In [1]:
import pandas as pd
import numpy as np
from env import conn
import matplotlib as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
%matplotlib inline
cs = conn.cursor()

In [2]:
cs.execute("SELECT * FROM sandbox.cm_sandbox.lc_links_joins_forecast")

In [3]:
query_result = cs.fetchall()

In [4]:
column_names = [col[0] for col in cs.description]

In [5]:
cs.close()

True

In [6]:
df = pd.DataFrame(query_result, columns=column_names)

In [7]:
df.head()

,DATE,CHANNEL,LOYALTY_PLAN_COMPANY,CUMULATIVE_JOINS
0,2023-11-04,LLOYDS,Stonegate Group,27
1,2023-11-12,LLOYDS,Stonegate Group,30
2,2023-11-02,LLOYDS,Stonegate Group,27
3,2023-10-26,LLOYDS,Stonegate Group,12
4,2023-10-22,LLOYDS,Stonegate Group,6
